<a href="https://colab.research.google.com/github/PedroBritodSa/NLP-Resume-job-description-scan/blob/main/NLP_project_resume_scan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Resume Scan and WordCloud
- In this notebook, we will implement a simple NLP structure to scan resumes in docx or PDF format and compare with a job description
- The code is structured in functions to facilitate usage and it is necessary to upload the documents to see it working.  

In [19]:
pip install docx2txt

In [20]:
pip install PyPDF2

- Loading some important libraries


In [21]:
import docx2txt #library to transform docx file into txt

from PyPDF2 import PdfReader #library to transform pdf file into txt

import os #Make the check for the type of document

from sklearn.feature_extraction.text import CountVectorizer #Simple vectorizer

from sklearn.metrics.pairwise import cosine_similarity #library to search simality in text

import string #to remove punctuations

import re #regular expression library

import pandas as pd

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS # Set of stopwords to be removed

from wordcloud import WordCloud

import matplotlib.pyplot as plt

import plotly.express as px

import nltk

from nltk.tokenize import word_tokenize

from collections import Counter

nltk.download("punkt")


punc = string.punctuation # punctuations to be removed

stopwords = ENGLISH_STOP_WORDS # stopwords to be removed

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import files

upload = files.upload() #upload the files you want to analyse

In [ ]:
#Create a function to check if the file is pdf or docx and then read it

def file_check(docname):
  file_extension = os.path.splitext(docname)[-1].lower()
  if file_extension == ".docx":

    data = docx2txt.process(docname)

    return data

  elif file_extension == ".pdf":

    reader = PdfReader("Statement_BU.pdf")
    n_pages = len(reader.pages)
    page = reader.pages[n_pages-1]
    data = page.extract_text()

    return data

  else:
    return print("Invalid Type of file")



# Data Visualization

- First we will create a tool to  visualize more frequent words in the douments. To do so, we will use the wordcloud library.
- In order to analyse mostly the keywords, it is important to clean the unnecessary words, which we know as stopwords. So we will do it in the following functions

In [ ]:
#insert the name of the document to be analyzed

docname = ""

In [9]:
# Creating a function to visualize the most frequency words in the data
# docname must be changed by the name of the docx  that will be analized, for example: docname = "name.docx"
def wc_func(docname):

  data = file_check(docname)

  data = ' '.join([word for word in data.split() if word.lower() not in stopwords]) # removing stopwords

  wc = WordCloud(background_color="Black", repeat=False, width = 1200 , height = 600) # Setting up the wordcloud

  wc.generate(data) # applying the wc on the data

  plt.figure(figsize = (20,20))

  plt.axis("off") # hide axis

  plt.imshow(wc, interpolation="bilinear")

  plt.show()





In [ ]:
wc_func("-file-name-")

- We can also examine the words by their frequency in the text, which will be set up in the following function.

In [11]:


def most_freq_words(docname,number):

    data = file_check(docname)

    data = ''.join([char for char in data if char not in punc])


    data = ' '.join([word for word in data.split() if word.lower() not in stopwords]) # removing stopwords

    words = word_tokenize(data.lower())
    # Count word frequency
    word_count = Counter(words)

    # Get the most common words and their frequencies
    most_common_words = word_count.most_common(number)

    # Convert the Counter object to a DataFrame
    df = pd.DataFrame(most_common_words, columns=['Word', 'freq'])

    df = df.sort_values(by='freq', ascending=True)

    fig = px.bar(df, y='Word', x='freq', text_auto='.2s',
            title="Most Common Words", orientation = 'h')

    fig.show()


In [ ]:
most_freq_words("-file-name-",18)

# Scanning the Resume

- Finally, we can use the code sklearn library to create a count_matrix of our texts, and them calculate the similarity between them using the cosine similarity method

In [12]:
def resume_scan(your_resume, job_description):

  resume = file_check(your_resume) #Reading resume docx/pdf

  job_des = file_check(job_description) #Reading job description docx/pdf

  resume = ' '.join([word for word in resume.split() if word.lower() not in stopwords])

  job_des = ' '.join([word for word in job_des.split() if word.lower() not in stopwords])

  text = [resume, job_des] # Seting the two texts in a dataset

  cv = CountVectorizer() # Creating the function CountVectorizer

  count_matrix = cv.fit_transform(text) #Creating the count matrix

  #Print the cosine similarity scores

  print("\nSimilarity Scores:")

  print(cosine_similarity(count_matrix))

  print(" ")

  print(" ")

  print("\nMatch Rate of: " + str(your_resume) +" and " + str(job_description))

  print(" ")

  print(" ")

  matchrate = cosine_similarity(count_matrix)[0][1]

  matchpercen = round(matchrate*100,2) # rouded value

  print("Match percentage between your resume and the job description is " + str(matchpercen))


In [ ]:
resume_scan("Resume","job_description")